In [1]:
import tempfile
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

import skimage.measure
from tensorflow import keras

mobile_tflite_intp = tf.lite.Interpreter(model_path = "E:\\MTP\\cifar10-70.tflite")
mobile_tflite_intp.allocate_tensors()


(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

test_images = test_images.astype(np.int32)

test_images = test_images - 128

current_test_image = np.expand_dims(test_images[10], axis=0).astype(np.float32)

input_index = mobile_tflite_intp.get_input_details()[0]["index"]

mobile_tflite_intp.set_tensor(input_index, current_test_image)
mobile_tflite_intp.invoke()
output_class = mobile_tflite_intp.get_tensor(10)

#print(np.shape(output_class))

def det_scale(indx):
    dic = [d for d in mobile_tflite_intp.get_tensor_details() if d['index'] == indx]
    scl = dic[0]['quantization_parameters']['scales']
    return scl

def det_zeroPos(indx):
    dic = [d for d in mobile_tflite_intp.get_tensor_details() if d['index'] == indx]
    scl = dic[0]['quantization_parameters']['zero_points']
    return scl

In [2]:
def cifar_predict(img):
    
    # layer1------------------------------------------------------------
    
    test_data_instance = img

    act_scale = det_scale(8)
    act_zeroP = det_zeroPos(8)

    wt_scale = det_scale(9)
    wt_zeroP = det_zeroPos(9)

    bias_scale = det_scale(4)
    bias_zeroP = det_zeroPos(4)

    output_scale = det_scale(10)
    output_zeroP = det_zeroPos(10)

    wt_tensor = mobile_tflite_intp.get_tensor(9)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(4))

    '''
    print(act_scale)
    print(wt_scale)
    print(output_scale)
    '''

    scale_factor = act_scale * wt_scale / output_scale
    #print(scale_factor)

    #print(wt_zeroP)
    #print(bias_tensor[0])
    

    conv_res_1 = np.zeros((1, 30, 30, 32))
    maxpool_res_1 = np.zeros((1, 15, 15, 32)).astype(np.int32)

    for h in range(30):
        for w in range(30):

            act_inst = test_data_instance[0, h:h+3, w:w+3, :]

            for c in range(32):

                wt_sel = wt_tensor[c, :, :, :]
                sum2 = output_zeroP + scale_factor[c] * (np.sum(np.multiply(act_inst, wt_sel)) + bias_tensor[c])
                #print(sum2)
                conv_res_1[0, h, w, c] = sum2

    conv_res_1 = np.maximum(-127, conv_res_1)
    conv_res_1 = np.minimum(127, conv_res_1)
    conv_res_1 = np.round(conv_res_1)
    conv_res_1 = conv_res_1.astype(np.int32)

    #print(conv_res_1.shape)
    #print(conv_res_1)

    for c in range(32):
        maxpool_res_temp = skimage.measure.block_reduce(conv_res_1[0, :, :, c], (2,2), np.max)
        maxpool_res_1[0, :, :, c] = maxpool_res_temp

    #print(maxpool_res_1.shape)
    #print(maxpool_res_1)

    #print(np.amin(maxpool_res_1))
    #print(np.amax(maxpool_res_1))
    
    
    # layer2------------------------------------------------------------
    
    act_scale = det_scale(11)
    act_zeroP = det_zeroPos(11)

    wt_scale = det_scale(12)
    wt_zeroP = det_zeroPos(12)

    bias_scale = det_scale(3)
    bias_zeroP = det_zeroPos(3)

    output_scale = det_scale(13)
    output_zeroP = det_zeroPos(13)

    wt_tensor = mobile_tflite_intp.get_tensor(12)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(3))

    conv_op_size = 13
    maxpool_op_size = 6
    op_channels = 64

    conv_res_2 = np.zeros((1, conv_op_size, conv_op_size, op_channels))
    maxpool_res_2 = np.zeros((1, maxpool_op_size, maxpool_op_size, op_channels)).astype(np.int32)

    test_data_instance = maxpool_res_1

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    
    for h in range(conv_op_size):
        for w in range(conv_op_size):

            act_inst = test_data_instance[0, h:h+3, w:w+3, :]

            for c in range(op_channels):

                wt_sel = wt_tensor[c, :, :, :]

                acc = np.sum(np.multiply(act_inst, wt_sel))

                sum2 = output_zeroP + scale_factor[c] * (acc + bias_tensor[c]) - scale_factor[c] * act_zeroP * np.sum(wt_sel)
                #print(sum2)
                conv_res_2[0, h, w, c] = sum2

    conv_res_2 = np.maximum(-127, conv_res_2)
    conv_res_2 = np.minimum(127, conv_res_2)
    conv_res_2 = np.round(conv_res_2)
    conv_res_2 = conv_res_2.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)


    for c in range(op_channels):
        maxpool_res_temp = skimage.measure.block_reduce(conv_res_2[0, 0:12, 0:12, c], (2,2), np.max)
        maxpool_res_2[0, :, :, c] = maxpool_res_temp
        
    
    # layer3------------------------------------------------------------
    
        
    act_scale = det_scale(14)
    act_zeroP = det_zeroPos(14)

    wt_scale = det_scale(15)
    wt_zeroP = det_zeroPos(15)

    bias_scale = det_scale(2)
    bias_zeroP = det_zeroPos(2)

    output_scale = det_scale(16)
    output_zeroP = det_zeroPos(16)

    wt_tensor = mobile_tflite_intp.get_tensor(15)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(2))

    conv_op_size = 4
    maxpool_op_size = 2
    op_channels = 128

    conv_res_3 = np.zeros((1, conv_op_size, conv_op_size, op_channels))
    maxpool_res_3 = np.zeros((1, maxpool_op_size, maxpool_op_size, op_channels)).astype(np.int32)

    test_data_instance = maxpool_res_2

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    
    for h in range(conv_op_size):
        for w in range(conv_op_size):

            act_inst = test_data_instance[0, h:h+3, w:w+3, :]

            for c in range(op_channels):

                wt_sel = wt_tensor[c, :, :, :]

                acc = np.sum(np.multiply(act_inst, wt_sel))

                sum2 = output_zeroP + scale_factor[c] * (acc + bias_tensor[c]) - scale_factor[c] * act_zeroP * np.sum(wt_sel)
                #print(sum2)
                conv_res_3[0, h, w, c] = sum2

    conv_res_3 = np.maximum(-126, conv_res_3)
    conv_res_3 = np.minimum(127, conv_res_3)
    conv_res_3 = np.round(conv_res_3)
    conv_res_3 = conv_res_3.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)


    for c in range(op_channels):
        maxpool_res_temp = skimage.measure.block_reduce(conv_res_3[0, :, :, c], (2,2), np.max)
        maxpool_res_3[0, :, :, c] = maxpool_res_temp
        
        
    # fc1------------------------------------------------------------

    act_scale = det_scale(18)
    act_zeroP = det_zeroPos(18)

    wt_scale = det_scale(19)
    wt_zeroP = det_zeroPos(19)

    bias_scale = det_scale(5)
    bias_zeroP = det_zeroPos(5)

    output_scale = det_scale(20)
    output_zeroP = det_zeroPos(20)

    wt_tensor = mobile_tflite_intp.get_tensor(19)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(5))

    conv_op_size = 128

    fc_res_1 = np.zeros((1, conv_op_size))

    test_data_instance = maxpool_res_3.flatten()

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    for c in range(conv_op_size):
     
        act_inst = test_data_instance
        wt_sel = wt_tensor[c]

        acc = np.sum(np.multiply(act_inst, wt_sel))

        sum2 = output_zeroP + scale_factor[0] * (acc + bias_tensor[c]) - scale_factor[0] * act_zeroP * np.sum(wt_sel)
        #print(sum2)
        fc_res_1[0, c] = sum2

    fc_res_1 = np.maximum(-126, fc_res_1)
    fc_res_1 = np.minimum(127, fc_res_1)
    fc_res_1 = np.round(fc_res_1)
    fc_res_1 = fc_res_1.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)

    
    
    # fc2------------------------------------------------------------

    act_scale = det_scale(20)
    act_zeroP = det_zeroPos(20)

    wt_scale = det_scale(21)
    wt_zeroP = det_zeroPos(21)

    bias_scale = det_scale(6)
    bias_zeroP = det_zeroPos(6)

    output_scale = det_scale(22)
    output_zeroP = det_zeroPos(22)

    wt_tensor = mobile_tflite_intp.get_tensor(21)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(6))

    conv_op_size = 84

    fc_res_2 = np.zeros((1, conv_op_size))

    test_data_instance = fc_res_1.flatten()

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    
    for c in range(conv_op_size):
     
        act_inst = test_data_instance
        wt_sel = wt_tensor[c]

        acc = np.sum(np.multiply(act_inst, wt_sel))

        sum2 = output_zeroP + scale_factor[0] * (acc + bias_tensor[c]) - scale_factor[0] * act_zeroP * np.sum(wt_sel)
        #print(sum2)
        fc_res_2[0, c] = sum2

    fc_res_2 = np.maximum(-126, fc_res_2)
    fc_res_2 = np.minimum(127, fc_res_2)
    fc_res_2 = np.round(fc_res_2)
    fc_res_2 = fc_res_2.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)
    
    
    # fc3------------------------------------------------------------

    act_scale = det_scale(22)
    act_zeroP = det_zeroPos(22)

    wt_scale = det_scale(23)
    wt_zeroP = det_zeroPos(23)

    bias_scale = det_scale(7)
    bias_zeroP = det_zeroPos(7)

    output_scale = det_scale(24)
    output_zeroP = det_zeroPos(24)

    wt_tensor = mobile_tflite_intp.get_tensor(23)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(7))

    conv_op_size = 10

    fc_res_3 = np.zeros((1, conv_op_size))

    test_data_instance = fc_res_2.flatten()

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    for c in range(conv_op_size):
     
        act_inst = test_data_instance
        wt_sel = wt_tensor[c]

        acc = np.sum(np.multiply(act_inst, wt_sel))

        sum2 = output_zeroP + scale_factor[0] * (acc + bias_tensor[c]) - scale_factor[0] * act_zeroP * np.sum(wt_sel)
        #print(sum2)
        fc_res_3[0, c] = sum2

    fc_res_3 = np.maximum(-128, fc_res_3)
    fc_res_3 = np.minimum(127, fc_res_3)
    fc_res_3 = np.round(fc_res_3)
    fc_res_3 = fc_res_3.astype(np.int32)
    
    return np.argmax(fc_res_3)

In [3]:
current_test_image = np.expand_dims(test_images[100], axis=0)

print(cifar_predict(current_test_image))

4


In [4]:
correct = 0
start_range = 100
end_range = 110
diff = end_range - start_range

for i in range(100, 110):
    current_test_image = np.expand_dims(test_images[i], axis=0)
    
    correct_label = test_labels[i][0]
    print("correct : ", correct_label)
    
    predicted_class = cifar_predict(current_test_image)
    print("predicted : ", predicted_class)
    
    
    print("----")
    if (correct_label == predicted_class):
        correct += 1
        
print("Accuracy = ", correct/diff * 100, "%")

correct :  4
predicted :  4
----
correct :  5
predicted :  7
----
correct :  6
predicted :  6
----
correct :  3
predicted :  3
----
correct :  1
predicted :  1
----
correct :  1
predicted :  1
----
correct :  3
predicted :  3
----
correct :  6
predicted :  6
----
correct :  8
predicted :  8
----
correct :  7
predicted :  7
----
Accuracy =  90.0 %


In [5]:
import math

A_sign_long_list = []
W_sign_long_list = []
A_val_long_list = []
W_val_long_list = []

A_sign_short_list = []
A_val_short_list = []
W_sign_short_list = []
W_val_short_list = []

bin =[0, 0, 0, 0]


def fill_short_vector_binary(sign_short_list, val_short_list, value_A_W):

    sign_short_list.clear();
    val_short_list.clear();

    for i in range(7):
        if (value_A_W & 1):
            sign_short_list.append(1)
            val_short_list.append(i)
        
        value_A_W = value_A_W >> 1;
   

    if (value_A_W & 1):
        sign_short_list.append(-1)
        val_short_list.append(7)
    

    #print(sign_short_list);
    #print(val_short_list);

def fill_short_vector_booth(sign_short_list, val_short_list, value_A_W): 

    sign_short_list.clear()
    val_short_list.clear()

    set = [0, 0, 0, 0]

    set[0] = (value_A_W & 0x3) << 1
    set[1] = (value_A_W >> 1) & 0x7
    set[2] = (value_A_W >> 3) & 0x7
    set[3] = (value_A_W >> 5) & 0x7

    for i in range(4):
        if (booth_lut(set[i]) != 0):
            termsA = booth_lut(set[i]) * pow(2, 2 * i)
            termsA = int(math.log(abs(termsA), 2))
            
            if (booth_lut(set[i]) > 0):
                sign_short_list.append(1)
            else:
                sign_short_list.append(-1)
            
            val_short_list.append(termsA)

            #sign_short_list.append(if(booth_lut(set[i]) > 0 else 1 : -1))
            
        
    #print(sign_short_list)
    #print(val_short_list)


def test_array_result():
    
    s_test_size = len(A_sign_long_list)
    accu = 0

    
    
    
    for i in range(s_test_size):
        exp_res = A_val_long_list[i] + W_val_long_list[i]
        sign_res = A_sign_long_list[i] * W_sign_long_list[i]

        accu += sign_res * pow(2, exp_res);
    

    return accu;
    


def laconic_simulator_result():
    
    
    #print("called")
    # laconic-simulator -------------------
    
    s_test_size = len(A_sign_long_list)
   #print("width =", s_test_size)
    accu23 = 0
    
    #word_length = 100

    itersteps = math.ceil(s_test_size / 16.0)
   #print(itersteps)
    laststep_len = (s_test_size % 16)
   #print(laststep_len)
   #print("---")

    for iter_inst in range(itersteps):

        start_index = 16*iter_inst
        end_index = min(16*iter_inst + 15, s_test_size-1)
       #print("---")
       #print(start_index, ":", end_index)
       #print("---")
    
    

        exp_add = np.zeros(16).astype(int)
        exp_sign = np.zeros(16).astype(int)

        decoder_array = np.zeros((16, 15)).astype(int)
        
        #print(decoder_array)


        for i_x1 in range(start_index, end_index + 1):
            exp_res = A_val_long_list[i_x1] + W_val_long_list[i_x1]
            exp_add[i_x1%16] = exp_res
            sign_res = A_sign_long_list[i_x1] * W_sign_long_list[i_x1]
            exp_sign[i_x1%16] = sign_res

        for i_x2 in range(start_index, end_index + 1):
            decoder_array[i_x2%16][int(exp_add[i_x2%16])] = 1


       #print("---")
       #print(decoder_array)
       #print("---")
       #print(exp_sign)
       #print("---")
        hist_input = np.zeros((16, 15)).astype(int)
        hist_input = decoder_array * exp_sign[:, np.newaxis]
        #print(hist_input)

        #decoder_out = decoder_array.sum(axis=0)
        #print(decoder_out)
       #print("---")
       #print("hist_input_pos")
        hist_input_pos = np.maximum(hist_input, 0)
        #print(hist_input_pos)

       #print("---")
       #print("hist_input_neg")
        hist_input_neg = np.minimum(hist_input, 0)
        hist_input_neg[hist_input < 0] = 1
       #print(hist_input_neg)


        histogram_out_pos = np.zeros(15)
        histogram_out_pos = histogram_out_pos.astype(int)
        histogram_out_neg = np.zeros(15)
        histogram_out_neg = histogram_out_neg.astype(int)
       #print("---")

        for r in range(15):

            pos_vec = hist_input_pos[:, r]
            histogram_out_pos[r] = GPC_count(pos_vec)
            #print("histogram_out_pos[",  r, "] :", histogram_out_pos[r])

       #print("---")

        for r in range(15):
            neg_vec = hist_input_neg[:, r]
            histogram_out_neg[r] = GPC_count(neg_vec)
            #print("histogram_out_neg[",  r, "] :", histogram_out_neg[r])


        GPC_out = np.zeros(16).astype(int)
        GPC_out = histogram_out_pos - histogram_out_neg

       #print("GPC_out", GPC_out)

        alignment_result = np.zeros(16).astype(int)

        for r in range(15):
            alignment_result[r] = GPC_out[r] << r
           #print("alignment_result[", r, "] =", alignment_result[r])


       #print("---")
        mac_res = np.sum(alignment_result)
       #print("MAC result = ", np.sum(alignment_result))

        #print("pos Hist")
        #print(histogram_out_pos)
        #print("neg Hist")
        #print(histogram_out_pos)
        #print(exp_sign)
       #print("---")
        #print(np.multiply(exp_sign, histogram_out))

        accu23 += mac_res
        #print("accu23 =", accu23)
        
   #print("MAC result = ", accu23)
    return accu23

        # laconic-simulator end -------------------


    
    
def booth_lut(in_val): 
    
    if (in_val == 0):
        return 0
    elif (in_val == 1):
        return 1
    elif (in_val == 2):
        return 1     
    elif (in_val == 3): 
        return 2      
    elif (in_val == 4):  
        return -2     
    elif (in_val == 5):  
        return -1 
    elif (in_val == 6):  
        return -1 
    else:
        return 0 

def encode_len_binary(A):
    termsA = 0

    for i_x in range(8):
        if (A & 1):
            termsA += 1

        A = A >> 1

    return termsA

def encode_len_booth(value_A_W):

    termsA = 0;

    set = [0, 0, 0, 0]

    set[0] = (value_A_W & 0x3) << 1;
    set[1] = (value_A_W >> 1) & 0x7;
    set[2] = (value_A_W >> 3) & 0x7;
    set[3] = (value_A_W >> 5) & 0x7;

    
    for i_x in range(4):
        if (booth_lut(set[i_x]) != 0):
            termsA += 1;

    return termsA;

def min_lane(A, W):

    A_binary = 0 
    A_booth = 0
    W_binary = 0
    W_booth = 0

    A_binary = encode_len_binary(A);
    A_booth = encode_len_booth(A);
    W_binary = encode_len_binary(W);
    W_booth = encode_len_booth(W);

    min_comb = [0, 0, 0, 0]

    min_comb[0] = A_binary * W_binary;
    min_comb[1] = A_binary * W_booth;
    min_comb[2] = A_booth * W_binary;
    min_comb[3] = A_booth * W_booth;


    min_index = min_comb.index(min(min_comb))
    #bin[min_index] += 1

    return(min_index)

def min_len_populate(A_in, W_in):

    path_to_take = min_lane(A_in, W_in)

    if (path_to_take == 0):
        fill_short_vector_binary(A_sign_short_list, A_val_short_list, A_in)
        fill_short_vector_binary(W_sign_short_list, W_val_short_list, W_in)
    elif (path_to_take == 1):
        fill_short_vector_binary(A_sign_short_list, A_val_short_list, A_in)
        fill_short_vector_booth(W_sign_short_list, W_val_short_list, W_in)
    elif (path_to_take == 2):
        fill_short_vector_booth(A_sign_short_list, A_val_short_list, A_in)
        fill_short_vector_binary(W_sign_short_list, W_val_short_list, W_in)
    else:
        fill_short_vector_booth(A_sign_short_list, A_val_short_list, A_in)
        fill_short_vector_booth(W_sign_short_list, W_val_short_list, W_in)

    w_len = len(W_sign_short_list)
    a_len = len(A_sign_short_list)

    for w_run in range(w_len):
        for a_run in range(a_len):

            W_sign_long_list.append(W_sign_short_list[w_run])
            W_val_long_list.append(W_val_short_list[w_run])

            A_sign_long_list.append(A_sign_short_list[a_run])
            A_val_long_list.append(A_val_short_list[a_run])

    

# ---------------------------------------------------
def input_A_W(A_subset, W_subset):
    
    A_subset_inst = A_subset.flatten()
    W_subset_inst = W_subset.flatten()
    
    A_sign_long_list.clear()
    W_sign_long_list.clear()
    A_val_long_list.clear()
    W_val_long_list.clear()


   #print("A_subset_inst :", A_subset_inst)
   #print("W_subset_inst :", W_subset_inst)

    for i in range(len(A_subset_inst)):
        min_len_populate(A_subset_inst[i], W_subset_inst[i])
        #print(min_lane(A_subset_inst[i], W_subset_inst[i]))
        #print("----")




   #print("A_sign_long_list :\t", A_sign_long_list)
   #print("A_val_long_list :\t", A_val_long_list)
   #print("W_sign_long_list :\t", W_sign_long_list)
   #print("W_val_long_list :\t", W_val_long_list)

    #print(A_subset_inst)
    #print(W_subset_inst)
    

    #print(type(A_subset_inst[0]))


def GPC_count(inp_vec):

    #return np.sum(inp_vec) #exact result
    #return 0 #exact result

    
    return ((inp_vec[0] | inp_vec[1] | inp_vec[2]) + 
            (inp_vec[3] | inp_vec[4]) + 
            (inp_vec[5] | inp_vec[6]) + 
            (inp_vec[7] | inp_vec[8]) + 
            (inp_vec[9] | inp_vec[10]) + 
            (inp_vec[11] | inp_vec[12]) + 
            (inp_vec[13] | inp_vec[14] | inp_vec[15]))
    
    
    #return (inp_vec[0] + inp_vec[1] + inp_vec[2] + inp_vec[3] + inp_vec[4] + inp_vec[5] + inp_vec[6] + inp_vec[7] + inp_vec[8] + inp_vec[9] + inp_vec[10] + inp_vec[11] + inp_vec[12] + inp_vec[13] + inp_vec[14])
    

            

    #return np.sum(inp_vec)




In [6]:
def cifar_predict_laconic(img):
    
    # layer1------------------------------------------------------------
    
    test_data_instance = img

    act_scale = det_scale(8)
    act_zeroP = det_zeroPos(8)

    wt_scale = det_scale(9)
    wt_zeroP = det_zeroPos(9)

    bias_scale = det_scale(4)
    bias_zeroP = det_zeroPos(4)

    output_scale = det_scale(10)
    output_zeroP = det_zeroPos(10)

    wt_tensor = mobile_tflite_intp.get_tensor(9)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(4))

    '''
    print(act_scale)
    print(wt_scale)
    print(output_scale)
    '''

    scale_factor = act_scale * wt_scale / output_scale
    #print(scale_factor)

    #print(wt_zeroP)
    #print(bias_tensor[0])
    

    conv_res_1 = np.zeros((1, 30, 30, 32))
    maxpool_res_1 = np.zeros((1, 15, 15, 32)).astype(np.int32)

    for h in range(30):
        for w in range(30):

            act_inst = test_data_instance[0, h:h+3, w:w+3, :]

            for c in range(32):

                wt_sel = wt_tensor[c, :, :, :]
                
                # software -------------------------
                
                #acc = np.sum(np.multiply(act_inst, wt_sel))
                
                
                #laconic ---------------------------
                
                input_A_W(act_inst, wt_sel)
                acc = laconic_simulator_result()
                
                #-----------------------------------
                
                sum2 = output_zeroP + scale_factor[c] * (acc + bias_tensor[c])
                #print(sum2)
                conv_res_1[0, h, w, c] = sum2
                
                #print("h =", h, "w =", w, "c =", c, " - done")

    conv_res_1 = np.maximum(-127, conv_res_1)
    conv_res_1 = np.minimum(127, conv_res_1)
    conv_res_1 = np.round(conv_res_1)
    conv_res_1 = conv_res_1.astype(np.int32)

    #print(conv_res_1.shape)
    #print(conv_res_1)

    for c in range(32):
        maxpool_res_temp = skimage.measure.block_reduce(conv_res_1[0, :, :, c], (2,2), np.max)
        maxpool_res_1[0, :, :, c] = maxpool_res_temp

    #print(maxpool_res_1.shape)
    #print(maxpool_res_1)

    #print(np.amin(maxpool_res_1))
    #print(np.amax(maxpool_res_1))
    
    print("layer1_done")
    
    
    # layer2------------------------------------------------------------
    
    act_scale = det_scale(11)
    act_zeroP = det_zeroPos(11)

    wt_scale = det_scale(12)
    wt_zeroP = det_zeroPos(12)

    bias_scale = det_scale(3)
    bias_zeroP = det_zeroPos(3)

    output_scale = det_scale(13)
    output_zeroP = det_zeroPos(13)

    wt_tensor = mobile_tflite_intp.get_tensor(12)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(3))

    conv_op_size = 13
    maxpool_op_size = 6
    op_channels = 64

    conv_res_2 = np.zeros((1, conv_op_size, conv_op_size, op_channels))
    maxpool_res_2 = np.zeros((1, maxpool_op_size, maxpool_op_size, op_channels)).astype(np.int32)

    test_data_instance = maxpool_res_1

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    
    for h in range(conv_op_size):
        for w in range(conv_op_size):

            act_inst = test_data_instance[0, h:h+3, w:w+3, :]

            for c in range(op_channels):

                wt_sel = wt_tensor[c, :, :, :]

                # software -------------------------
                
                #acc = np.sum(np.multiply(act_inst, wt_sel)
                
                
                #laconic ---------------------------
                
                input_A_W(act_inst, wt_sel)
                acc = laconic_simulator_result()
                
                #-----------------------------------

                sum2 = output_zeroP + scale_factor[c] * (acc + bias_tensor[c]) - scale_factor[c] * act_zeroP * np.sum(wt_sel)
                #print(sum2)
                conv_res_2[0, h, w, c] = sum2

    conv_res_2 = np.maximum(-127, conv_res_2)
    conv_res_2 = np.minimum(127, conv_res_2)
    conv_res_2 = np.round(conv_res_2)
    conv_res_2 = conv_res_2.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)


    for c in range(op_channels):
        maxpool_res_temp = skimage.measure.block_reduce(conv_res_2[0, 0:12, 0:12, c], (2,2), np.max)
        maxpool_res_2[0, :, :, c] = maxpool_res_temp
        
    print("layer2_done")
    
    # layer3------------------------------------------------------------
    
        
    act_scale = det_scale(14)
    act_zeroP = det_zeroPos(14)

    wt_scale = det_scale(15)
    wt_zeroP = det_zeroPos(15)

    bias_scale = det_scale(2)
    bias_zeroP = det_zeroPos(2)

    output_scale = det_scale(16)
    output_zeroP = det_zeroPos(16)

    wt_tensor = mobile_tflite_intp.get_tensor(15)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(2))

    conv_op_size = 4
    maxpool_op_size = 2
    op_channels = 128

    conv_res_3 = np.zeros((1, conv_op_size, conv_op_size, op_channels))
    maxpool_res_3 = np.zeros((1, maxpool_op_size, maxpool_op_size, op_channels)).astype(np.int32)

    test_data_instance = maxpool_res_2

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    
    for h in range(conv_op_size):
        for w in range(conv_op_size):

            act_inst = test_data_instance[0, h:h+3, w:w+3, :]

            for c in range(op_channels):

                wt_sel = wt_tensor[c, :, :, :]

                # software -------------------------
                
                #acc = np.sum(np.multiply(act_inst, wt_sel)
                
                
                #laconic ---------------------------
                
                input_A_W(act_inst, wt_sel)
                acc = laconic_simulator_result()
                
                #-----------------------------------

                
                
                sum2 = output_zeroP + scale_factor[c] * (acc + bias_tensor[c]) - scale_factor[c] * act_zeroP * np.sum(wt_sel)
                #print(sum2)
                conv_res_3[0, h, w, c] = sum2

    conv_res_3 = np.maximum(-126, conv_res_3)
    conv_res_3 = np.minimum(127, conv_res_3)
    conv_res_3 = np.round(conv_res_3)
    conv_res_3 = conv_res_3.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)


    for c in range(op_channels):
        maxpool_res_temp = skimage.measure.block_reduce(conv_res_3[0, :, :, c], (2,2), np.max)
        maxpool_res_3[0, :, :, c] = maxpool_res_temp
        
    print("layer3_done")
    
    # fc1------------------------------------------------------------

    act_scale = det_scale(18)
    act_zeroP = det_zeroPos(18)

    wt_scale = det_scale(19)
    wt_zeroP = det_zeroPos(19)

    bias_scale = det_scale(5)
    bias_zeroP = det_zeroPos(5)

    output_scale = det_scale(20)
    output_zeroP = det_zeroPos(20)

    wt_tensor = mobile_tflite_intp.get_tensor(19)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(5))

    conv_op_size = 128

    fc_res_1 = np.zeros((1, conv_op_size))

    test_data_instance = maxpool_res_3.flatten()

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    for c in range(conv_op_size):
     
        act_inst = test_data_instance
        wt_sel = wt_tensor[c]

        # software -------------------------
                
        #acc = np.sum(np.multiply(act_inst, wt_sel)


        #laconic ---------------------------

        input_A_W(act_inst, wt_sel)
        acc = laconic_simulator_result()

        #-----------------------------------

        sum2 = output_zeroP + scale_factor[0] * (acc + bias_tensor[c]) - scale_factor[0] * act_zeroP * np.sum(wt_sel)
        #print(sum2)
        fc_res_1[0, c] = sum2

    fc_res_1 = np.maximum(-126, fc_res_1)
    fc_res_1 = np.minimum(127, fc_res_1)
    fc_res_1 = np.round(fc_res_1)
    fc_res_1 = fc_res_1.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)

    
    
    # fc2------------------------------------------------------------

    act_scale = det_scale(20)
    act_zeroP = det_zeroPos(20)

    wt_scale = det_scale(21)
    wt_zeroP = det_zeroPos(21)

    bias_scale = det_scale(6)
    bias_zeroP = det_zeroPos(6)

    output_scale = det_scale(22)
    output_zeroP = det_zeroPos(22)

    wt_tensor = mobile_tflite_intp.get_tensor(21)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(6))

    conv_op_size = 84

    fc_res_2 = np.zeros((1, conv_op_size))

    test_data_instance = fc_res_1.flatten()

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    
    for c in range(conv_op_size):
     
        act_inst = test_data_instance
        wt_sel = wt_tensor[c]

        # software -------------------------
                
        #acc = np.sum(np.multiply(act_inst, wt_sel)


        #laconic ---------------------------

        input_A_W(act_inst, wt_sel)
        acc = laconic_simulator_result()

        #-----------------------------------

        sum2 = output_zeroP + scale_factor[0] * (acc + bias_tensor[c]) - scale_factor[0] * act_zeroP * np.sum(wt_sel)
        #print(sum2)
        fc_res_2[0, c] = sum2

    fc_res_2 = np.maximum(-126, fc_res_2)
    fc_res_2 = np.minimum(127, fc_res_2)
    fc_res_2 = np.round(fc_res_2)
    fc_res_2 = fc_res_2.astype(np.int32)

    #print(conv_res_2.shape)
    #print(conv_res_2)
    
    
    # fc3------------------------------------------------------------

    act_scale = det_scale(22)
    act_zeroP = det_zeroPos(22)

    wt_scale = det_scale(23)
    wt_zeroP = det_zeroPos(23)

    bias_scale = det_scale(7)
    bias_zeroP = det_zeroPos(7)

    output_scale = det_scale(24)
    output_zeroP = det_zeroPos(24)

    wt_tensor = mobile_tflite_intp.get_tensor(23)
    bias_tensor = np.array(mobile_tflite_intp.get_tensor(7))

    conv_op_size = 10

    fc_res_3 = np.zeros((1, conv_op_size))

    test_data_instance = fc_res_2.flatten()

    scale_factor = act_scale * wt_scale / output_scale
    scale_factor_b = bias_scale / output_scale
    
    for c in range(conv_op_size):
     
        act_inst = test_data_instance
        wt_sel = wt_tensor[c]

        # software -------------------------
                
        acc = np.sum(np.multiply(act_inst, wt_sel))


        #laconic ---------------------------

        #input_A_W(act_inst, wt_sel)
        #acc = laconic_simulator_result()

        #-----------------------------------

        sum2 = output_zeroP + scale_factor[0] * (acc + bias_tensor[c]) - scale_factor[0] * act_zeroP * np.sum(wt_sel)
        #print(sum2)
        fc_res_3[0, c] = sum2

    fc_res_3 = np.maximum(-128, fc_res_3)
    fc_res_3 = np.minimum(127, fc_res_3)
    fc_res_3 = np.round(fc_res_3)
    fc_res_3 = fc_res_3.astype(np.int32)
    
    return np.argmax(fc_res_3)

In [13]:
def accuracy_check(start_range, end_range):

    correct = 0
    diff = end_range - start_range

    for i in range(start_range, end_range):
        current_test_image = np.expand_dims(test_images[i], axis=0)

        correct_label = test_labels[i][0]
        #print("correct : ", correct_label)

        predicted_class = cifar_predict(current_test_image)
        #print("predicted : ", predicted_class)


        #print("----")
        if (correct_label == predicted_class):
            correct += 1

    print(correct, "correct within",  start_range, "to", (end_range - 1))

accuracy_check(100, 110)

9 correct within 100 to 109


In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(3) as p:
        print(p.map(f, [1, 2, 3]))

In [ ]:
def accuracy_check_2(start_range):

    correct = 0
    diff = end_range - start_range

    for i in range(start_range, start_range + 10):
        current_test_image = np.expand_dims(test_images[i], axis=0)

        correct_label = test_labels[i][0]
        #print("correct : ", correct_label)

        predicted_class = cifar_predict_laconic(current_test_image)
        #print("predicted : ", predicted_class)


        #print("----")
        if (correct_label == predicted_class):
            correct += 1

    print(correct, "correct within",  start_range, "to", (start_range + 9))

accuracy_check_2(40)